<h1>snRNA-seq re-analysis (Schirmer et al., Nature, 2019)

In [ ]:
# load libraries
library(devtools)
library(Seurat)
library(dplyr)
library(Matrix)
library(ggplot2)

In [3]:
library(tidyverse)
library(Seurat)
library(Polychrome)

In [ ]:
# import UMI matrix from CellRanger
matrix_dir = "XXX" # inster directory here
data_raw = Read10X(data.dir = matrix_dir)
data = CreateSeuratObject(counts = data_raw, project = "snRNA-seq", min.cells = 3, min.features = 200)

In [ ]:
# calculate the percentage of mitochondrial features
mito.features = grep(pattern = "^MT-", x = rownames(x = data), value = TRUE)
percent.mito = Matrix::colSums(x = GetAssayData(object = data, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = data, slot = 'counts'))
data[['percent.mito']] = percent.mito # store information

In [ ]:
# filter out cells that have unique feature counts over 5000 or less than 200
data = subset(data, subset = nFeature_RNA > 500 & nCount_RNA > 1000)

In [ ]:
# Normalizing the data
data = NormalizeData(data, normalization.method = "LogNormalize", scale.factor = 1e4)

In [ ]:
# detection of variable genes across the single cells
data <- FindVariableFeatures(data, selection.method = 'mean.var.plot', mean.cutoff = c(0.0125, 3), dispersion.cutoff = c(0.5, Inf))

In [ ]:
# remove MT genes
markers.remove <- grep(pattern = "^MT-",  x = rownames(data), value = TRUE)

In [ ]:
data@assays$RNA@var.features <- data@assays$RNA@var.features[!(data@assays$RNA@var.features%in%markers.remove)]

In [ ]:
# scaling the data and removing unwanted sources of variation
data = ScaleData(data)

In [ ]:
# perform linear dimensional reduction
data = RunPCA(data, features = VariableFeatures(data), verbose = FALSE)
# examine and visualize PCA results a few different ways
print(x = data[['pca']], dims = 1:5, nfeatures = 5, projected = FALSE)

In [ ]:
# get batches based on cell names
samples_batches = sapply(colnames(GetAssayData(data, slot = "counts")),
                      FUN=function(x){substr(x,18,19)})

In [ ]:
# turn to numbers and add cell names to them
samples_batches = as.numeric(as.factor(samples_batches))
names(samples_batches) = colnames(GetAssayData(data, slot = "counts"))

In [ ]:
# Plot genes per cell
# Complexity per technical batch
complexity.per.cell = apply(GetAssayData(object = csf, slot = "counts"),
                             2, function(x) sum(x>0))
plot(complexity.per.cell ~ jitter(samples_batches,2))

In [ ]:
sample.effect = samples_batches
data = AddMetaData(data, sample.effect, "sample.effect")

In [ ]:
for (i in 1:length(x = data.list)) {
    data.list[[i]] = NormalizeData(object = data.list[[i]], verbose = FALSE)
    data.list[[i]] = FindVariableFeatures(object = data.list[[i]], 
        selection.method = "vst", nfeatures = 2000, verbose = FALSE)
}

In [ ]:
reference.list <- csf.list
csf.anchors <- FindIntegrationAnchors(object.list = reference.list, dims = 1:30)

In [ ]:
data.integrated = IntegrateData(anchorset = data.anchors, dims = 1:30)

In [ ]:
# switch to integrated assay
DefaultAssay(object = data.integrated) <- "integrated"

In [ ]:
# Scaling the data and removing unwanted sources of variation
data.integrated <- ScaleData(object = data.integrated)

In [ ]:
# perform linear dimensional reduction
data.integrated <- RunPCA(object = data.integrated, npcs = 30, verbose = FALSE)
# examine and visualize PCA results a few different ways
print(x = data.integrated[['pca']], dims = 1:5, nfeatures = 5, projected = FALSE)
VizDimLoadings(object = data.integrated, dims = 1:2)
DimPlot(object = data.integrated)
DimPlot(object = data.integrated, group.by="sample.effect")

In [ ]:
# dimensional reduction and clustering
data = FindNeighbors(data, dims = 1:16)
data = FindClusters(data, resolution = 0.8)

In [ ]:
# perform non-linear dimensional reduction (tSNE)
data = RunTSNE(data, dims = 1:16)

In [ ]:
# save data
write.table(csf@meta.data, 'meta.data.tsv', sep='\t')
saveRDS(data.integrated, file = "snRNA-seq.integrated.rds")
saveRDS(data, file = "snRNA-seq.rds")

In [21]:
# visualize data
data = readRDS("snRNA-seq.rds")

data(palette36) # color palette for clusters
colpal = palette36[1:33]
colpal = colpal %>% as.vector()

ERROR: Error: kann Vektor der Größe 331.4 MB nicht allozieren


In [20]:
# tSNE plot and clustering
Idents(data) <- "RNA_snn_res.0.8"
DimPlot(data, reduction = 'tsne', cols = colpal, label = F, label.size = 5, pt.size = 1) + 
  theme_void() + NoLegend()
ggsave('output/tsne.png', plot = last_plot(), scale = 1, dpi = 900)
DimPlot(data, reduction = 'tsne', cols = colpal, label = T, label.size = 5, pt.size = 1) + 
  theme_void()
ggsave('output/tsne.pdf', plot = last_plot(), scale = 1)

ERROR: Error: Cannot find 'tsne' in this Seurat object


In [16]:
# signature genes for clusters (from original paper)
sig_genes = c('SYT1', 'SLC17A7', 'GAD2', 'AQP4', 'PLP1', 'P2RY12', 'CSF1R', 'CSF1', 'IL34')
for (i in 1:length(sig_genes)){
    FeaturePlot(data, features = sig_genes[i], label.size = 5, cols = c("lightgrey", "red3"), pt.size = 1.5) +
        theme_void() + NoLegend() + theme(title = element_blank())
    ggsave(paste0('output/', 'tsne_', tolower(sig_genes[i]), '.png'), plot = last_plot(), scale = 1, dpi = 900, bg = "transparent")
}

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [17]:
# CSF1R and ligands in tsne
goi = c('CSF1R', 'IL34', 'CSF1') # define genes of interest
for (i in 1:length(goi)){
    FeaturePlot(data, features = goi[i], label.size = 5, cols = c("lightgrey", "red3"), pt.size = 1.5) +
        theme_void() + NoLegend() + theme(title = element_blank())
    ggsave(paste0('output/', 'tsne_', tolower(goi[i]), '.png'), plot = last_plot(), scale = 1, dpi = 900)
}

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [18]:
# cluster-specific expression of CSF1R, CSF1 and IL34
goi = c('CSF1R', 'IL34', 'CSF1') # define genes of interest
for (i in 1:length(goi)){
    Idents(data) <- "RNA_snn_res.0.8"
    DotPlot(data, features = goi[i], dot.scale = 3, cols = c('red3', 'red3'), split.by = "diagnosis") +
      theme(text=element_text(size=6,  family="sans"),
        axis.text.x = element_text(angle=45, vjust=1, hjust=1, size=6, family="sans"),
        axis.text.y = element_text(size=6, family="sans", hjust = 1, vjust = 0.35),
        axis.line = element_line(colour = 'black', size = 0.2),
        axis.ticks = element_line(colour = 'black', size = 0.2)
        )
    ggsave(paste0('output/', 'dotplot_', tolower(goi[i]), '_msctrl.pdf'), plot = last_plot(), width = 6, height = 16, units = "cm")
}